<a href="https://colab.research.google.com/github/whikwon/colab_test/blob/master/191024_%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%82%AC%EC%9D%B4%EC%96%B8%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install pydicom

     |████████████████████████████████| 7.1MB 2.8MB/s 


## Load dicom sample 

In [0]:
import os

import pydicom
import numpy as np
import matplotlib.pyplot as plt

In [0]:
data_dir = "/content/drive/My Drive/1910_빅데이터사이언스실무자양성과정/dataset"
sample_dcm_path = os.path.join(data_dir, "0002.DCM")

In [0]:
dcm_data = pydicom.read_file("/content/drive/My Drive/1910_빅데이터사이언스실무자양성과정/dataset/0002.DCM")

FileNotFoundError: ignored

In [0]:
dcm_data

(0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY', 'SINGLE PLANE', 'SINGLE A']
(0008, 0016) SOP Class UID                       UI: X-Ray Angiographic Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.4.3.321890.19960124.162922.29
(0008, 0020) Study Date                          DA: '19941013'
(0008, 0030) Study Time                          TM: '141917'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'XA'
(0008, 0070) Manufacturer                        LO: ''
(0008, 0080) Institution Name                    LO: ''
(0008, 0081) Institution Address                 ST: ''
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1030) Study Description                   LO: ''
(0008, 1050) Performing Physician's Name         PN: ''
(0008, 2110) Lossy Image Compression (Retired)   CS: '01'
(0008, 2112)  Source Image Sequence   1 item(s) ---- 
   (0008, 1150) Referen

In [0]:
dcm_data[(0x0008, 0x0008)]

(0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY', 'SINGLE PLANE', 'SINGLE A']

In [0]:
pixel_arr = dcm_data.pixel_array

In [0]:
pixel_arr.shape

(96, 512, 512)

In [0]:
for i in range(len(pixel_arr)):
  plt.figure(figsize=(10, 10))
  plt.imshow(pixel_arr[i])
  plt.show()

In [0]:
curve_data = dcm_data[(0x05000, 0x03000)]

In [0]:
curve_data.value

b'\xff\x00\xff\x00\xff\x00\xff\x00\xff\x00\xff\x00\xff\x00\xff\x00\xfd\x00\xee\x00\xdf\x00\xd1\x00\xc5\x00\xbb\x00\xb2\x00\xa9\x00\xa3\x00\x9d\x00\x99\x00\x95\x00\x8f\x00\x89\x00\x86\x00\x83\x00\x81\x00\x80\x00\x7f\x00\x7f\x00~\x00\x7f\x00}\x00~\x00~\x00|\x00{\x00y\x00w\x00v\x00u\x00s\x00r\x00r\x00q\x00q\x00q\x00o\x00n\x00n\x00l\x00m\x00m\x00m\x00m\x00m\x00m\x00l\x00m\x00l\x00k\x00l\x00k\x00j\x00i\x00g\x00e\x00f\x00h\x00h\x00h\x00j\x00i\x00k\x00m\x00n\x00o\x00n\x00m\x00m\x00n\x00m\x00m\x00n\x00m\x00l\x00m\x00n\x00o\x00m\x00k\x00k\x00j\x00i\x00i\x00j\x00k\x00j\x00j\x00l\x00m\x00m\x00m\x00m\x00m\x00m\x00n\x00p\x00q\x00p\x00o\x00o\x00n\x00n\x00n\x00o\x00n\x00n\x00o\x00o\x00p\x00m\x00j\x00i\x00g\x00f\x00f\x00f\x00h\x00i\x00j\x00k\x00l\x00n\x00m\x00p\x00q\x00p\x00q\x00p\x00p\x00o\x00o\x00n\x00m\x00m\x00k\x00j\x00j\x00i\x00h\x00i\x00k\x00l\x00n\x00o\x00q\x00s\x00t\x00r\x00r\x00p\x00o\x00n\x00l\x00j\x00j\x00j\x00k\x00m\x00n\x00m\x00l\x00k\x00j\x00j\x00j\x00j\x00j\x00k\x00k\x00k\x00l\x00l\x00j

## Pytorch Dataset

- https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [0]:
import os
import glob

import torch
from torch.utils.data import Dataset, DataLoader

In [0]:
class CustomDataset(Dataset):
  
  def __init__(self, data_dir):
    self.data_dir = data_dir
    self.filelist = glob.glob(os.path.join(data_dir, "*.DCM"))
    
  def __len__(self):
    return len(self.filelist)
    
  def __getitem__(self, idx):
    dcm_data = pydicom.read_file(self.filelist[idx])
    imgs = dcm_data.pixel_array
    assert imgs.ndim == 3
    
    # see: https://pytorch.org/docs/stable/nn.html#conv3d
    imgs = np.expand_dims(imgs, axis=1)
    assert imgs.shape == (96, 1, 512, 512)
    
    # augmentation    
    return imgs

In [0]:
dataset = CustomDataset(data_dir)

In [0]:
dataset.filelist

['/content/drive/My Drive/1910_빅데이터사이언스실무자양성과정/dataset/0002.DCM',
 '/content/drive/My Drive/1910_빅데이터사이언스실무자양성과정/dataset/0003.DCM']

In [0]:
dataloader = DataLoader(dataset, batch_size=2)

In [0]:
samples = iter(dataloader).next()

(96, 1, 512, 512)
(96, 1, 512, 512)


In [0]:
samples.shape

torch.Size([2, 96, 1, 512, 512])